In [ ]:
import re, os

import numpy as np


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os
import pickle

import pandas as pd

from sat_func import SATFunc, sat_eval

In [ ]:
#plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')
plt.style.use('default')

out_dir = 'sat_results/'
results_set = {}
for fl in sorted(os.listdir(out_dir)):
    if 'pkl' not in fl:
        continue
    #if '_150_' in fl:
    #    continue
    with open(os.path.join(out_dir,fl),'rb') as fp:
        import_data = pickle.load(fp)
        if type(import_data) == list:
            results = import_data
        else:
            results = import_data['results']
    dim = int('_'.join(fl.split('_')[-2:-1]))
    algo = '_'.join(fl.split('_')[0:2])
    algo = algo.replace('single_cma','CMA')

    algo = algo.replace('posthoc_cma','post hoc')
    algo = algo.replace('staged_cma','staged')
    algo = algo.replace('onlinewin_cma','online')

    results_set[(algo,dim,fl)] = results

In [ ]:
[_ for _ in results_set.keys() if '_1.0_1.0_' not in _[2]],[_ for _ in results_set.keys() if '_150_' in _[2]]

In [ ]:
plt.figure(dpi=250,figsize=(6, 3))
dfs = [] 
for name in sorted(results_set):

    res = results_set[name]
    df = pd.DataFrame(res,columns=['res','base','oracle'])
    df['name'] = name[-1]
    df['method'] = name[0]
    df['iter'] = name[1]
    df['res'] = np.array(df[['res','base']]).min(axis=1)
    dfs.append(df)
    #mean_v = df.groupby('dim').mean()
    #min_v = df.groupby('dim').min()
df = pd.concat(dfs)
base_min_v = df.min(axis=0)

mean_df = df.drop('name',axis=1).groupby(['method','iter']).mean().reset_index()
min_df = df.drop('name',axis=1).groupby(['method','iter']).min().reset_index()

std_df = df.drop('name',axis=1).groupby(['method','iter']).std().reset_index()
cnt_df= df.drop('name',axis=1).groupby(['method','iter']).count().reset_index()

import seaborn as sns
from collections import defaultdict
with sns.color_palette('plasma', n_colors=len(df.method.unique())):
    span_v = base_min_v.base-base_min_v.oracle

    convert_f = lambda x: (base_min_v.oracle-x)/span_v+1
    for method in df.method.unique():
        mean_p = convert_f(mean_df[mean_df.method == method].res)
        diff = 1.96*std_df[std_df.method == method].res/np.sqrt(cnt_df[cnt_df.method == method].res)

        ub = convert_f(mean_df[mean_df.method == method].res-diff)
        lb = convert_f(mean_df[mean_df.method == method].res+diff)

        label = name[0]#.upper()
        ls_lookup = defaultdict(lambda: 'dashdot')
        ls_lookup['cma'] = '-'
        ls_lookup['post hoc'] = '--'
        ls_lookup['online'] = 'dotted'

        ls = ls_lookup[method.lower()]
        ms_lookup = defaultdict(lambda: '^')
        ms_lookup['CMA'] = '*'
        ms_lookup['online'] = 'v'
        ms_lookup['post hoc'] = 'x'
        ms = ms_lookup[method.lower()]
        label_p = method[0].upper() + method[1:]

        plt.plot(mean_df[mean_df.method == method].iter,mean_p,label=label_p,ls=ls,marker=ms,lw=2)
        plt.fill_between(mean_df[mean_df.method == method].iter,lb,ub,alpha=0.15)
    #plt.plot(mean_df[mean_df.method == method].iter,mean_df[mean_df.method == method].base,label="Initialization",c='k',ls='--')
    #plt.plot(mean_df[mean_df.method == method].iter,mean_df[mean_df.method == method].oracle,label="Oracle",c='r',ls='--')

plt.xlabel('Function Evaluations')
plt.ylabel('Quality of minima \n (0 is initialization, 1 is oracle)')
plt.title('Optimizing Stereo Matching Parameters')
plt.legend()
plt.tight_layout()
plt.legend(ncol=4,loc='lower right',fancybox=True,shadow=False,framealpha=0.5)
#plt.ylim(bottom=0)
plt.xlim(right=250)
plt.tight_layout()
plt.savefig('lol_sat_newest.pdf')

In [ ]:
df[(df.iter == 150) & (df.method =='online')]

In [ ]:
mean_df# 1.074519

In [ ]:
min_df

In [ ]:
#plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')
plt.style.use('default')

results_Algo = {}
for fl in sorted(os.listdir(out_dir)):
    if 'pkl' not in fl:
        continue
    #if '_150_' in fl:
    #    continue
    with open(os.path.join(out_dir,fl),'rb') as fp:
        import_data = pickle.load(fp)
        if type(import_data) == list:
            results = import_data
        else:
            results = import_data['results']
    dim = int('_'.join(fl.split('_')[-2:-1]))
    algo = '_'.join(fl.split('_')[0:2])
    algo = algo.replace('single_cma','CMA')

    algo = algo.replace('posthoc_cma','post hoc')
    algo = algo.replace('staged_cma','staged')
    algo = algo.replace('onlinewin_cma','online')
    res_arr = np.array(results)
    min_idx = np.argsort(res_arr[:,0])[0]
    results_Algo[(algo,dim,fl)] = (res_arr[min_idx,0],tuple(import_data['clusters'][min_idx]))

In [ ]:
sorted([(v[0],v[1],k[0],k[1]) for k,v in results_Algo.items()])